<a href="https://colab.research.google.com/github/Elwing-Chou/ml0804/blob/master/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0804/raw/master/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0804/raw/master/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df